In [ ]:
import pandas as pd
from sqlalchemy import create_engine

db="sqlite:///top2020.db"
engine = create_engine(db,echo=False)

In [66]:
df_entries = pd.read_sql_table('entries',engine)
df_albums = pd.read_sql_table('albums',engine,index_col='id')
df_users = pd.read_sql_table('users',engine,index_col='id')
df_genres = pd.read_sql_table('genres',engine,index_col='id')

In [67]:
def build_entries():
    entries = pd.merge(df_entries, df_albums, left_on='album_id', right_on='id')
    entries = pd.merge(entries, df_genres, left_on='genre_id', right_on='id')

    return entries

entries = build_entries()
entries.head()

,id,name_x,user_id,album_id,position,score,name_y,genre_id,name
0,1,déhà - how to despise humanity in 7 lessons an...,1,1,1,30.000000,déhà - how to despise humanity in 7 lessons an...,24,Experimental
1,1082,déhà - how to despise humanity in 7 lessons an...,64,1,10,19.772727,déhà - how to despise humanity in 7 lessons an...,24,Experimental
2,26,melted bodies - enjoy yourself,6,26,4,27.413793,melted bodies - enjoy yourself,24,Experimental
3,541,melted bodies-enjoy yourself,33,26,2,29.137931,melted bodies - enjoy yourself,24,Experimental
4,1087,melted bodies - enjoy yourself,64,26,15,14.090909,melted bodies - enjoy yourself,24,Experimental


In [68]:
def format_entries(entries):

    map = {
        'id':'entry_id',
        'user_id':'user_id',
        'album_id':'album_id',
        'name_y':'album',
        'genre_id':'genre_id',
        'name':'genre',
        'position':'position',
        'score':'score'
    }

    drop_cols = (x for x in entries.columns if x not in map.keys())

    entries.drop(drop_cols, axis=1, inplace=True)
    entries.rename(map, axis=1, inplace=True)
    entries.set_index('entry_id', drop=False, inplace=True)

    return entries

entries = format_entries(entries)
entries.head()


,entry_id,user_id,album_id,position,score,album,genre_id,genre
entry_id,,,,,,,,
1,1,1,1,1,30.000000,déhà - how to despise humanity in 7 lessons an...,24,Experimental
1082,1082,64,1,10,19.772727,déhà - how to despise humanity in 7 lessons an...,24,Experimental
26,26,6,26,4,27.413793,melted bodies - enjoy yourself,24,Experimental
541,541,33,26,2,29.137931,melted bodies - enjoy yourself,24,Experimental
1087,1087,64,26,15,14.090909,melted bodies - enjoy yourself,24,Experimental


In [108]:
def build_album_results(entries):

    aggfunc = {
        'position': ['mean','min','max'],
        'score': ['count','sum','mean','max','min']
    }

    album_stats = pd.pivot_table(entries, index = ['genre_id','genre','album_id','album'], values = ['score','position'], aggfunc=aggfunc)

    album_ranking = album_stats.rank(method='dense',ascending=False)[('score','sum')]
    album_genre_ranking = album_stats.groupby('genre_id').rank(method='dense',ascending=False)[('score','sum')]

    album_results = pd.merge(album_stats, album_ranking, left_index=True, right_index=True)
    album_results = pd.merge(album_results, album_genre_ranking, left_index=True, right_index=True)

    album_results.reset_index(inplace=True)

    return album_results

album_results = build_album_results(entries)
album_results.head()

genre_id        genre album_id                                  album  \
                                                                          
0        1  Heavy Metal       19        cirith ungol - forever in black   
1        1  Heavy Metal      180             dynazty - the dark delight   
2        1  Heavy Metal      205              lonewolf - division hades   
3        1  Heavy Metal      213               avatar - hunter gatherer   
4        1  Heavy Metal      337  spirit adrift enlightened in eternity   

  position                  score_x                                   \
       max       mean   min   count        max       mean        min   
0     26.0  10.000000   3.0       4  27.619048  16.278736   5.000000   
1     20.0  20.000000  20.0       1  13.620690  13.620690  13.620690   
2     30.0  15.500000   1.0       2  30.000000  17.500000   5.000000   
3     13.0   9.833333   1.0       6  30.000000  18.032581  13.928571   
4     30.0  30.000000  30.0       1   5.000000   5.000000   5.000000   

              score_y score  
          sum     sum   sum  
0   65.114943    64.0   2.0  
1   13.620690   267.0   7.0  
2   35.000000   148.0   3.0  
3  108.195489    31.0   1.0  
4    5.000000   306.0  11.0

In [109]:
def format_album_results(album_results):

    album_results.columns = album_results.columns.map('|'.join).str.strip('|')

    map = {
        'score_y|sum':'rank',
        'album_id':'album_id',
        'album':'album',
        'score|sum':'genre_rank',
        'genre_id':'genre_id',
        'genre':'genre',
        'score_x|count':'nb_votes',
        'score_x|sum':'total_score',
        'score_x|mean':'mean_score',
        'score_x|max':'highest_score',
        'score_x|min':'lowest_score',
        'position|mean':'mean_position',
        'position|min':'highest_position',
        'position|max':'lowest_position'
    }

    album_results = album_results.reindex(columns=map.keys())

    drop_cols = (x for x in album_results.columns if x not in map.keys())
    album_results.drop(drop_cols, axis=1, inplace=True)

    album_results.rename(map, axis=1, inplace=True)

    album_results.set_index('album_id', inplace=True, drop=True)

    album_results.sort_values('rank', inplace=True)

    return album_results

album_results = format_album_results(album_results)

album_results.head()

,rank,album,genre_rank,genre_id,genre,nb_votes,total_score,mean_score,highest_score,lowest_score,mean_position,highest_position,lowest_position
album_id,,,,,,,,,,,,,
35,1.0,the ocean - phanerozoic ii : mesozoic / cenozoic,1.0,3,Prog Metal,15,324.158629,21.610575,30.000000,5.862069,8.933333,1.0,29.0
31,2.0,napalm death - throes of joy in the jaws of de...,1.0,21,Crust - Grind,15,283.355725,18.890382,29.137931,7.380952,10.666667,2.0,26.0
245,3.0,kvelertak - splid,1.0,39,Metal'n'Roll,12,257.413283,21.451107,29.137931,5.000000,6.500000,2.0,20.0
81,4.0,loudblast - manifesto,1.0,4,Death Metal,13,228.271757,17.559366,30.000000,5.000000,10.076923,1.0,24.0
57,5.0,oranssi pazuzu - mestarin kynsi,1.0,8,Post-Black Metal,11,222.261967,20.205633,30.000000,5.000000,10.545455,1.0,30.0


In [77]:

def extend_entries(entries, album_results):

    album_keepcols = [
        'album_id',
        'total_score'
    ]
    album_dropcols = (x for x in album_results.columns if x not in album_keepcols)
    album_scores = album_results.drop(album_dropcols, axis=1)

    full_entries = pd.merge(entries, album_scores, on='album_id')

    full_entries = pd.merge(full_entries, df_users, left_on='user_id', right_on='id')

    full_entries.set_index('entry_id', inplace=True)
    cols = [
        'user_id',
        'name',
        'album_id',
        'album',
        'genre_id',
        'genre',
        'top_size',
        'position',
        'score',
        'total_score'
    ]

    full_entries = full_entries.reindex(columns=cols)

    return full_entries

full_entries = extend_entries(entries, album_results)
full_entries.head()

,user_id,name,album_id,album,genre_id,genre,top_size,position,score,total_score
entry_id,,,,,,,,,,
1,1,Denis Porcin,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,4,1,30.000000,49.772727
2,1,Denis Porcin,2,sepultura - quadra,2,Thrash Metal,4,2,21.666667,220.968801
3,1,Denis Porcin,3,jonathan hulten - chants from another place,29,Neofolk - DarkFolk,4,3,13.333333,43.333333
4,1,Denis Porcin,4,tomorrow's rain - hollow,11,Doom Metal,4,4,5.000000,32.578370
1082,64,Ugo Petropoulos,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,23,10,19.772727,49.772727


In [83]:
def compute_entry_stats(df):

   df['pop_score'] = df['score'] * df['total_score'] / 1000
   # idea for a future implementation
   # df['pop_score'] = df['score'] * ( df['total_score'] - df['total_score'].quantile(q=0.666) ) / 1000
   df['edgyness'] = df['top_size'] / df['pop_score']

   df.sort_values('entry_id', inplace=True)

full_entries = compute_entry_stats(full_entries)
full_entries.head()

,user_id,name,album_id,album,genre_id,genre,top_size,position,score,total_score,pop_score,edgyness
entry_id,,,,,,,,,,,,
1,1,Denis Porcin,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,4,1,30.000000,49.772727,1.493182,2.678843
2,1,Denis Porcin,2,sepultura - quadra,2,Thrash Metal,4,2,21.666667,220.968801,4.787657,0.835482
3,1,Denis Porcin,3,jonathan hulten - chants from another place,29,Neofolk - DarkFolk,4,3,13.333333,43.333333,0.577778,6.923077
4,1,Denis Porcin,4,tomorrow's rain - hollow,11,Doom Metal,4,4,5.000000,32.578370,0.162892,24.556170
1082,64,Ugo Petropoulos,1,déhà - how to despise humanity in 7 lessons an...,24,Experimental,23,10,19.772727,49.772727,0.984143,23.370598


In [111]:
def compute_user_stats(full_entries):
    
    user_genres = pd.pivot_table(full_entries, index=['name'], columns=['genre'], values=['score'], aggfunc=['sum'])

    aggfunc = {
       'pop_score': 'sum',
       'edgyness': 'mean'
    }
    user_edgyness = pd.pivot_table(full_entries, index = ['name'], values=['pop_score','edgyness'], aggfunc=aggfunc)

    user_edgyness.sort_values(('edgyness'), ascending=False, inplace=True)

    return user_genres,user_edgyness

user_genres, user_edgyness = compute_user_stats(full_entries)
user_edgyness.head(20)

,edgyness,pop_score
name,,
Alexis Hérissant,196.445040,22.179822
David Vermalen,171.166050,14.068766
Pagus Ström,165.329756,29.986856
Francky Rockois,154.728884,42.634397
David Chardiet,142.191275,72.554854
"Ronan ""youpi crêpier"" Lesclache",141.319131,47.902311
David Decaudin,141.243020,14.856602
Moland Fengkov,140.492984,47.054770
Arnaud Dehon,139.625061,14.059001


In [82]:
user_genres.head(20)

sum                                \
                                       score                                 
genre               Alternative Rock / Metal Avant-Garde Metal Black Metal   
name                                                                         
Alex KM                                  NaN         41.034483  104.137931   
Alexandre Ardisson                       NaN               NaN   95.625000   
Alexis Hérissant                   17.068966         41.896552  102.586207   
Amduscias Baal                           NaN               NaN   18.888889   
Anthony Pwl                              NaN               NaN         NaN   
Arnaud Dehon                             NaN               NaN  167.857143   
Arno STROBL                              NaN               NaN         NaN   
Arsonist Tohliam                         NaN               NaN   45.000000   
Arthur Bourson                           NaN               NaN   47.500000   
Brett Caldas-Lima                        NaN               NaN         NaN   
Cam Iel                                  NaN         12.142857   32.857143   
Caroline d'Avout                   25.833333               NaN         NaN   
Corinne Royer béril                      NaN               NaN   18.793103   
Cretch                                   NaN               NaN   17.500000   
Cyril Deparis                            NaN         24.655172  358.620690   
Cyrille Druet                            NaN         22.105263   37.631579   
David Chardiet                           NaN               NaN   22.857143   
David Daz                          91.379310               NaN   19.655172   
David Decaudin                           NaN               NaN         NaN   
David Gibert                             NaN               NaN         NaN   

                                                      \
                                                       
genre               Blues - Blues Rock Crust - Grind   
name                                                   
Alex KM                      25.689655           NaN   
Alexandre Ardisson                 NaN           NaN   
Alexis Hérissant                   NaN     43.620690   
Amduscias Baal                     NaN           NaN   
Anthony Pwl                        NaN           NaN   
Arnaud Dehon                       NaN           NaN   
Arno STROBL                        NaN     27.727273   
Arsonist Tohliam                   NaN           NaN   
Arthur Bourson                     NaN           NaN   
Brett Caldas-Lima                  NaN           NaN   
Cam Iel                            NaN     15.714286   
Caroline d'Avout                   NaN           NaN   
Corinne Royer béril          47.931034           NaN   
Cretch                             NaN           NaN   
Cyril Deparis                      NaN           NaN   
Cyrille Druet                      NaN           NaN   
David Chardiet               23.750000           NaN   
David Daz                          NaN           NaN   
David Decaudin               14.375000           NaN   
David Gibert                       NaN           NaN   

                                                                      \
                                                                       
genre               Dark Synth / Dungeon synth Death Metal Deathcore   
name                                                                   
Alex KM                                    NaN   30.000000       NaN   
Alexandre Ardisson                         NaN         NaN       NaN   
Alexis Hérissant                           NaN  156.896552       NaN   
Amduscias Baal                             NaN         NaN       NaN   
Anthony Pwl                                NaN         NaN       NaN   
Arnaud Dehon                               NaN   61.904762       NaN   
Arno STROBL                                NaN    5.000000       NaN   
Arsonist Tohliam               

In [90]:
def compute_genre_stats(df):

    genre_stats = pd.pivot_table(df, index=['genre'], values=['nb_votes','total_score'], aggfunc=['sum'])
    genre_stats.sort_values(('sum','nb_votes'), ascending=False, inplace=True)

    genre_stats['weight'] = genre_stats[('sum','total_score')] * 100.0 / genre_stats[('sum','total_score')].sum()

    return genre_stats

genre_stats = compute_genre_stats(album_results)
genre_stats.head(20)

sum                  weight
                         nb_votes  total_score           
genre                                                    
Black Metal                   202  3616.887197  17.940909
Death Metal                   111  1781.570175   8.837154
Doom Metal                     96  1588.970247   7.881797
Thrash Metal                   72  1265.888615   6.279209
Stoner - Psych                 72  1231.781665   6.110028
Prog Metal                     61  1130.650069   5.608383
Post-Black Metal               52   993.615057   4.928646
Sludge                         41   676.207837   3.354206
Experimental                   36   714.073309   3.542030
Prog - Psychedelic Rock        35   619.524190   3.073037
Crust - Grind                  34   640.352851   3.176353
Melodeath                      28   499.418599   2.477275
Post-Metal                     27   523.486842   2.596661
Avant-Garde Metal              24   421.540276   2.090974
Alternative Rock / Metal       21   376.134409   1.865746
Heavy Metal                    21   334.677053   1.660104
Folk - Pagan Metal             19   292.195791   1.449384
HxC                            19   324.903530   1.611625
Blues - Blues Rock             17   301.257868   1.494335
Post-Hardcore                  17   318.112462   1.577939